### Finding the intersect of pythons ranges

I have a lot data in paginated memory blocks. Each page is generally of a block size but can vary due to inserts and deletes.

My challenge: Determine whether the contents of a page needs to be loaded to answer a users query for slice.

Here's a simplistic example:

```
page = range(500,700,1)
user = range(10,1000,30)
userdata = []
for row in intercept(page,user):
      userdata.append(row)
```

Here is the solution:

In [2]:

import math


In [3]:
def intercept(A,B):
    assert isinstance(A, range)
    if A.step < 0: # turn the range around
        A = range(A.stop, A.start, abs(A.step))
    assert isinstance(B, range)
    if B.step < 0:  # turn the range around
        B = range(B.stop, B.start, abs(B.step))
    
    boundaries = [A.start, A.stop, B.start, B.stop]
    boundaries.sort()
    a,b,c,d = boundaries
    if [A.start, A.stop] in [[a,b],[c,d]]:
        return range(0) # then there is no intercept
    # else: The inner range (subset) is b,c, limited by the first shared step.
    A_start_steps = math.ceil((b - A.start) / A.step)
    A_start = A_start_steps * A.step + A.start

    B_start_steps = math.ceil((b - B.start) / B.step)
    B_start = B_start_steps * B.step + B.start

    intersection = set(range(A_start, c, A.step)).intersection(set(range(B_start, c, B.step)))
    if not intersection:
        return range(0)
    start = min(intersection)
    end = max(intersection)
    intersection.remove(start)
    step = min(intersection) - start
    
    return range(start, end+1, step)


In [4]:


A = range(500,700,3)
B = range(520,700,3)
C = range(10,1000,30)

assert intercept(A,C) == range(0)
assert intercept(B,C) == range(520,671,30)


In [5]:


A = range(500_000, 700_000, 1)
B = range(10, 10_000_000, 1000)

assert len(intercept(A,B)) == 200


In [6]:
%timeit intercept(A,B)


12.2 ms ± 511 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


That's fast enough for me.